# Ex
### Exercise 8.1
- 2, 6, 19, 20, 25

### Exercise 8.2
- 4, 6, 13, 14, 15, 16, 17

### Exercise 8.3
- 3, 6, 7

### Exercise 8.4
- 3, 6, 7, 8, 9

# Com Ex
### Computer Exercise 8.1
- 2, 6, 7

### Computer Exercise 8.2
- 1, 12(a), 12(b)

### Computer Exercise 8.3
- 1(a), 7

### Computer Exercise 8.4
- 3

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [3]:
# 8.1 - 2

def dolittle_decomposition(A):
    n = A.shape[0]
    U = A.copy()
    L = np.zeros((n, n))
    
    for i in range(n):
        L[i, i] = 1
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= U[i, i:] * L[j, i]

    return L, U

def crout_decomposition(A):
    n = A.shape[0]
    L, U = dolittle_decomposition(A)
    LT = np.transpose(L)
    D = np.diag(np.diag(U))
    
    return L, D, LT

def verify_dolittle(A):
    L, U = dolittle_decomposition(A)
    
    print("L:\n", L)
    print("U:\n", U)
    print("LU:\n", L @ U)
    print("A:\n", A)
    if np.allclose(L @ U, A):
        print("pass")
    else:
        print("fail")
        
def verify_crout(A):
    L, D, LT = crout_decomposition(A)
    
    print("L:\n", L)
    print("D:\n", D)
    print("LT:\n", LT)
    print("LDT:\n", L @ D @ LT)
    print("A:\n", A)
    if np.allclose(L @ D @ LT, A):
        print("pass")
    else:
        print("fail")

A = np.array(
    [[5, 7, 6, 5], 
     [7, 10, 8, 7], 
     [6, 8, 10, 9], 
     [5, 7, 9, 10]]
    , dtype=float
)

verify_dolittle(A)
print()
verify_crout(A)

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
U:
 [[ 5.   7.   6.   5. ]
 [ 0.   0.2 -0.4  0. ]
 [ 0.   0.   2.   3. ]
 [ 0.   0.   0.   0.5]]
LU:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
D:
 [[5.  0.  0.  0. ]
 [0.  0.2 0.  0. ]
 [0.  0.  2.  0. ]
 [0.  0.  0.  0.5]]
LT:
 [[ 1.   1.4  1.2  1. ]
 [ 0.   1.  -2.   0. ]
 [ 0.   0.   1.   1.5]
 [ 0.   0.   0.   1. ]]
LDT:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass


In [3]:
# 8.1 - 6, 7

def build_tridiagonal(n, a, b, c):
    A = np.diag([a] * (n - 1), -1) + np.diag([b] * n) + np.diag([c] * (n - 1), 1)
    return A

def inverse_tridiagonal_naive(A):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape) 
    n = A.shape[0]
    
    L[0, 0] = 1
    U[0, :] = A[0, :]
    for i in range(1, n):
        L[i, i] = 1
        L[i, i - 1] = A[i, i - 1] / U[i - 1, i - 1]
        U[i, i] = A[i, i] - U[i - 1, i] * L[i, i - 1]
        if i < n - 1:
            U[i, i + 1] = A[i, i + 1]
        
    L_inv = np.eye(n, dtype=float)
    for i in range(1, n):
        for j in range(0, i):
            L_inv[i, j] = - L[i, i-1] * L_inv[i-1, j]
            
    U_inv = np.zeros((n, n), dtype=float)
    for i in range(n-1, -1, -1):
        U_inv[i, i] = 1.0 / U[i, i]
        for j in range(i+1, n):
            if i + 1 < n:
                U_inv[i, j] = - (U[i, i+1] * U_inv[i+1, j]) / U[i, i]
            else:
                U_inv[i, j] = 0.0
                
    A_inv = U_inv.dot(L_inv)
    return A_inv

def inverse_tridiagonal_thomas(A):
    n = A.shape[0]
    A_inv = np.zeros((n, n), dtype=float)
    I = np.eye(n)
    
    for i in range(n):
        A_inv[:, i] = solve_thomas(A, I[:, i])
    
    return A_inv

def solve_thomas(A, d):
    n = len(d)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    for i in range(n):
        b[i] = A[i][i]
        if i < n - 1:
            c[i] = A[i][i+1]
        if i > 0:
            a[i] = A[i][i-1]
    
    cp = np.zeros(n)
    dp = np.zeros(n)
    
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    
    for i in range(1, n):
        denom = b[i] - a[i] * cp[i - 1]
        if i < n - 1:
            cp[i] = c[i] / denom
        dp[i] = (d[i] - a[i] * dp[i - 1]) / denom
    
    x = np.zeros(n)
    x[-1] = dp[-1]
    for i in range(n - 2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i + 1]
    
    return x
        
def verify_inverse_tridiagonal(A, f):
    A_inv = f(A)
    print("A:\n", A)
    print("A_inv:\n", A_inv)
    print("A_inv @ A:\n", A_inv @ A)
    if np.allclose(A_inv @ A, np.eye(A.shape[0])):
        print("pass")
    else:
        print("fail")
        
verify_inverse_tridiagonal(build_tridiagonal(10, 1, -2, 1), inverse_tridiagonal_thomas)

A:
 [[-2  1  0  0  0  0  0  0  0  0]
 [ 1 -2  1  0  0  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0  0  0]
 [ 0  0  0  1 -2  1  0  0  0  0]
 [ 0  0  0  0  1 -2  1  0  0  0]
 [ 0  0  0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  0  0  1 -2  1]
 [ 0  0  0  0  0  0  0  0  1 -2]]
A_inv:
 [[-0.90909091 -0.81818182 -0.72727273 -0.63636364 -0.54545455 -0.45454545
  -0.36363636 -0.27272727 -0.18181818 -0.09090909]
 [-0.81818182 -1.63636364 -1.45454545 -1.27272727 -1.09090909 -0.90909091
  -0.72727273 -0.54545455 -0.36363636 -0.18181818]
 [-0.72727273 -1.45454545 -2.18181818 -1.90909091 -1.63636364 -1.36363636
  -1.09090909 -0.81818182 -0.54545455 -0.27272727]
 [-0.63636364 -1.27272727 -1.90909091 -2.54545455 -2.18181818 -1.81818182
  -1.45454545 -1.09090909 -0.72727273 -0.36363636]
 [-0.54545455 -1.09090909 -1.63636364 -2.18181818 -2.72727273 -2.27272727
  -1.81818182 -1.36363636 -0.90909091 -0.45454545]
 [-0.45454545 -0.90909091 -1.363

In [12]:
# 8.2 - 1

# a
A = np.array(
    [[1, 7], 
     [2, -5]]
)

A_eigvalue, A_eigenvector = sp.linalg.eig(A)

print(f"A: {A}")
print(f"eigen value: {A_eigvalue}")
print(f"eigen vector: {A_eigenvector}")
print()

# b
B = np.array(
    [[4, -7, 3, 2, 3],
     [1, 6, 11, -1, 2],
     [5, -5, -2, -4, 1],
     [9, -3, 1, 6, 5],
     [3, 2, 5, -5, 1]]
)

B_eigvalue, B_eigenvector = sp.linalg.eig(B)

print(f"B: {B}")
print(f"eigen value: {B_eigvalue}")
print(f"eigen vector: {B_eigenvector}")
print()

# c
def build_matrix_c(n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i <= j:
                A[i, j] = i / (j + 1)
            else:
                A[i, j] = (j + 1) / (i + 1)
    return A

C = build_matrix_c(12)

C_eigvalue, C_eigenvector = sp.linalg.eig(C)

print(f"C: {C}")
print(f"eigen value: {C_eigvalue}")
print(f"eigen vector: {C_eigenvector}")
print()

# d
def build_matrix_d(n):
    main_diag = 2 * np.ones(n)
    off_diag = -1 * np.ones(n - 1)
    A = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
    return A

D_1 = build_matrix_d(5)
D_2 = build_matrix_d(20)

D_1_eigvalue, D_1_eigenvector = sp.linalg.eig(D_1)
D_2_eigvalue, D_2_eigenvector = sp.linalg.eig(D_2)

print(f"D_1: {D_1}")
print(f"D_1 eigen value: {D_1_eigvalue}")
print(f"D_1 eigen vector: {D_1_eigenvector}")
for i, eig in enumerate(D_1_eigvalue):
    print(2 - 2*np.cos((i+1)*np.pi/(5+1)))
print()
print(f"D_2: {D_2}")
print(f"D_2 eigen value: {D_2_eigvalue}")
print(f"D_2 eigen vector: {D_2_eigenvector}")
for i, eig in enumerate(D_2_eigvalue):
    print(2 - 2*np.cos((i+1)*np.pi/(5+1)))
print()

# e
def build_matrix_e(n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if j >= i:
                A[i, j] = n - j
                
    return A

E = build_matrix_e(30)
E_eigvalue, E_eigenvector = sp.linalg.eig(E)

print(f"E: {E}")
print(f"E eigen value: {E_eigvalue}")
print(f"E eigen vector: {E_eigenvector}")
for i, eig in enumerate(E_eigvalue):
    print(1/(2 - 2*np.cos((2*i)*np.pi/(61))))
print()

A: [[ 1  7]
 [ 2 -5]]
eigen value: [ 2.79583152+0.j -6.79583152+0.j]
eigen vector: [[ 0.96863191 -0.66810749]
 [ 0.24849996  0.74406477]]

B: [[ 4 -7  3  2  3]
 [ 1  6 11 -1  2]
 [ 5 -5 -2 -4  1]
 [ 9 -3  1  6  5]
 [ 3  2  5 -5  1]]
eigen value: [ 5.26652176e+00+8.88267338j  5.26652176e+00-8.88267338j
  9.69214673e+00+0.j         -5.23387869e+00+0.j
  8.68843731e-03+0.j        ]
eigen vector: [[ 0.14369289+0.30174472j  0.14369289-0.30174472j -0.42786174+0.j
   0.64442482+0.j         -0.33812933+0.j        ]
 [ 0.58690827+0.j          0.58690827-0.j          0.2786041 +0.j
   0.46012634+0.j          0.0627388 +0.j        ]
 [-0.0581887 +0.38795358j -0.0581887 -0.38795358j -0.00436881+0.j
  -0.50587636+0.j         -0.18414327+0.j        ]
 [ 0.48164764-0.13469658j  0.48164764+0.13469658j -0.77970442+0.j
  -0.23966988+0.j         -0.18301298+0.j        ]
 [ 0.27377303+0.25469191j  0.27377303-0.25469191j  0.36243072+0.j
  -0.24422912+0.j          0.90240229+0.j        ]]

C: [[0.         0

C:\Users\hamas\AppData\Local\Temp\ipykernel_30168\93290875.py:95: RuntimeWarning: divide by zero encountered in scalar divide
  print(1/(2 - 2*np.cos((2*i)*np.pi/(61))))


In [15]:
# 8.2 - 12

def SVD(A):
    U, s, Vh = np.linalg.svd(A)
    S = np.zeros(A.shape)
    for i in range(len(s)):
        S[i, i] = s[i]
    
    print ("A:\n", A)
    print ("U:\n", U)
    print ("S:\n", S)
    print ("Vh:\n", Vh)

# a
A = np.array([[2, 1, -2]])
SVD(A)
print()

# b
B = np.array([[3], [4]])
SVD(B)
print()

A:
 [[ 2  1 -2]]
U:
 [[-1.]]
S:
 [[3. 0. 0.]]
Vh:
 [[-0.66666667 -0.33333333  0.66666667]
 [-0.33333333  0.93333333  0.13333333]
 [ 0.66666667  0.13333333  0.73333333]]

A:
 [[3]
 [4]]
U:
 [[ 0.6 -0.8]
 [ 0.8  0.6]]
S:
 [[5.]
 [0.]]
Vh:
 [[1.]]



In [ ]:
# 8.3 - 1
A = np.array(
    [[5, 4, 1, 1], 
     [4, 5, 1, 1], 
     [1, 1, 4, 2], 
     [1, 1, 2, 4]]
)

def power_method(A, x0, tol=1e-10, max_iter=1000):
    x = x0.astype(float)
    x /= np.linalg.norm(x)

    lambda_old = 0.0
    for _ in range(1, max_iter + 1):
        y = A @ x
        lambda_new = x @ y  
        norm_y = np.linalg.norm(y)
        x_new = y / norm_y

        if np.linalg.norm(x_new - x) < tol and abs(lambda_new - lambda_old) < tol:
            x = x_new
            lambda_old = lambda_new
            break

        x = x_new
        lambda_old = lambda_new

    return lambda_new, x

a, b = power_method(A, np.array([1, 1, 1, 1]))
print("a:", a)
print("b:", b)

eigval, eigvec = sp.linalg.eig(A)
print("eigval:", eigval)
print("eigvec:", eigvec)

def inverse_power_method(A, x0, tol=1e-10, max_iter=1000, shift=0.0):
    n, m = A.shape
    
    B = A - shift * np.eye(n)
    
    x = x0.astype(float)
    x /= np.linalg.norm(x)
    
    lambda_old = 0.0
    for _ in range(1, max_iter+1):
        y = np.linalg.solve(B, x)
        norm_y = np.linalg.norm(y)
        x_new = y / norm_y
        
        lambda_new = x_new @ (A @ x_new)
        
        if np.linalg.norm(x_new - x) < tol and abs(lambda_new - lambda_old) < tol:
            x = x_new
            lambda_old = lambda_new
            break
        
        x = x_new
        lambda_old = lambda_new
    
    return lambda_new, x



a: 10.000000000000002
b: [0.63245553 0.63245553 0.31622777 0.31622777]
eigval: [10.+0.j  1.+0.j  5.+0.j  2.+0.j]
eigvec: [[-6.32455532e-01 -7.07106781e-01  3.16227766e-01  1.00260275e-16]
 [-6.32455532e-01  7.07106781e-01  3.16227766e-01 -6.67394257e-17]
 [-3.16227766e-01 -9.25914416e-18 -6.32455532e-01 -7.07106781e-01]
 [-3.16227766e-01 -6.44544667e-17 -6.32455532e-01  7.07106781e-01]]


In [ ]:
# 8.3 - 7

A = np.array(
    [[-4, 14, 0],
     [-5, 13, 0],
     [-1, 0, 2]]
)

x0 = np.transpose(np.array([[1, 1, 1]]))

def accelerated_power_method(A, x0, tol=1e-10, max_iter=1000):
    x = x0.astype(float)
    x /= np.linalg.norm(x)

    lambda_old = 0.0
    for _ in range(1, max_iter + 1):
        y = A @ x
        lambda_new = x @ y  
        norm_y = np.linalg.norm(y)
        x_new = y / norm_y

        if np.linalg.norm(x_new - x) < tol and abs(lambda_new - lambda_old) < tol:
            x = x_new
            lambda_old = lambda_new
            break

        x = x_new
        lambda_old = lambda_new

    return lambda_new, x

[[1]
 [1]
 [1]]
